## Cover page placeholder

In [ ]:
# pip install python-constraint

# AI - Hiring Problem (CSP) 

## Scenario 1 
### Ciara knows Python, and only has funds to hire three more people.

It won't be possible to fill all of positions with just 3 hires, so Ciara will have to take on one of the Python positions.

In [2]:
from constraint import Problem

# Defines the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1"],
    "Peter": ["Python_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

problem = Problem()

# Defines the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# Defines the constraints
'''
IMPORTANT: Ciara only has the funds to hire 3 more people
She knows python and will fill one of the python positions
for this reason
'''
# Ciara will be taking a python position
def ciara_python_constraint(python_1):
    return python_1 == "Ciara"

# A person can only ocuppy one of the two available AI positions
def ai_constraint(ai_1, ai_2):
    return ai_1 != ai_2

# Jane is the only one qualified to ocuppy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people that will be hired (including Ciara)
def ciara_hiring_constraint(*positions):
    return len(set(positions)) <= 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*positions):
    hires_count = dict()
    for hire in positions:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Defines the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Adds the constraints to the problem
problem.addConstraint(ciara_python_constraint, ["Python_1"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(ciara_hiring_constraint, positions)
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)

# Solutions
solutions = problem.getSolutions()

# Removes duplicates
unique_solutions1 = []
for solution in solutions:
    # For positions with the same qualifications, sort the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"]])
    solution["AI_1"], solution["AI_2"] = ai_positions[0], ai_positions[1]
    
    # Adds to unique solutions if not present already
    if solution not in unique_solutions1:
        unique_solutions1.append(solution)

# Prints the solutions by position
for i, solution in enumerate(unique_solutions1):
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()

Solution 1:
Position AI_1: Anita
Position AI_2: Jim
Position Python_1: Ciara
Position Database: Jane
Position Python_2: Jane
Position Systems: Jim
Position Web: Anita

Solution 2:
Position AI_1: Jim
Position AI_2: Juan
Position Python_1: Ciara
Position Database: Jane
Position Python_2: Jane
Position Systems: Jim
Position Web: Juan



## Scenario 2
### Ciara and Juan become partners, with the additional funds they can now employ four more people but must employ another AI Engineer, so they need 2 Python Programmers, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.

This scenario will assume the following hiring conditions:

- Ciara and Juan will be considered for the positions they are qualified for.
- Solutions with Ciara OR Juan OR none of them are also going to be considered.

In [3]:
from constraint import Problem

# Defines the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1", "Python_2"],
    "Peter": ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Python_1", "Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_1", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

problem = Problem()

# Defines the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# # Defines the constraints
def python_constraint(python_1, python_2):
    return python_1 != python_2

def ai_constraint(ai_1, ai_2, ai_3):
    return ai_1 != ai_2 and ai_1 != ai_3 and ai_2 != ai_3

# Jane is the only one qualified to occupy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people that can be hired (Including Ciara and Juan)
def hiring_constraint(*args):
    hires_set = set(args)
    if "Ciara" in hires_set and "Juan" in hires_set:
        return len(hires_set) <= 6
    elif "Ciara" in hires_set or "Juan" in hires_set:
        return len(hires_set) <= 5
    else:
        return len(hires_set) == 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*args):
    hires_count = dict()
    for hire in args:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Defines the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Adds the constraints to the problem
problem.addConstraint(python_constraint, ["Python_1", "Python_2"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2", "AI_3"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(hiring_constraint, positions)
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)

# Gets and prints solutions
solutions = problem.getSolutions()

total_combinations = 0
unique_solutions2 = []
for solution in solutions:
    # For positions with the same qualifications, sorts the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"], solution["AI_3"]])
    solution["AI_1"], solution["AI_2"], solution["AI_3"] = ai_positions[0], ai_positions[1], ai_positions[2]

    python_positions = sorted([solution["Python_1"], solution["Python_2"]])
    solution["Python_1"], solution["Python_2"] = python_positions

    # Adds to unique solutions if not already present
    if solution not in unique_solutions2:
        unique_solutions2.append(solution)
        total_combinations+=1


# Prints the solutions by position
for i, solution in enumerate(unique_solutions2):   
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()

Solution 1:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Bruce
Position Web: Juan

Solution 2:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Bruce
Position Web: Anita

Solution 3:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Jim
Position Web: Juan

Solution 4:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Jim
Position Web: Anita

Solution 5:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Jane
Position Database: Jane
Position Systems: Bruce
Position Web: Juan

Solution 6:
Position AI_1: Anita
Position AI_2: Jim
P

In [4]:
# Initialize a dictionary to store the counts for each person
person_counts = dict()

# LoopS over the unique solutions
for solution in unique_solutions2:
    # GetS the set of hires in each solution
    hires_set = set(solution.values())
    # LoopS over the hires in the set
    for hire in hires_set:
        # IncrementS the count for the hire by one
        person_counts[hire] = person_counts.get(hire, 0) + 1

# DefineS a function to extract the count from an item
def get_count(item):
    return item[1]

# SortS the person counts by ascending order of values
person_counts = dict(sorted(person_counts.items(), key=get_count))

print(f"There is a total of {total_combinations} hiring combinations")
print()
# PrintS the counts for each person
for person, count in person_counts.items():
    if count >= total_combinations:
        print(f"{person} appears in ALL solutions")
    else:
        print(f"{person} appears in {count} solutions.")

print()

# PrintS solutions with specific hires
solutions_without_ciara_count = 0
solutions_without_juan_count = 0
solutions_without_neither_count = 0
solutions_with_both_count = 0

for i, solution in enumerate(unique_solutions2):
    if "Ciara" not in solution.values():
        solutions_without_ciara_count += 1
    if "Juan" not in solution.values():
        solutions_without_juan_count += 1
    if "Ciara" not in solution.values() and "Juan" not in solution.values():
        solutions_without_neither_count += 1
    if "Ciara" in solution.values() and "Juan" in solution.values():
        solutions_with_both_count += 1  
        
print(f"The number of solutions that Ciara is not in is: {solutions_without_ciara_count}")
print(f"The number of solutions that Juan is not in is: {solutions_without_juan_count}")
print(f"The number of solutions that neither is in is: {solutions_without_neither_count}")
print(f"The number of solutions with BOTH in it is: {solutions_with_both_count}")

There is a total of 77 hiring combinations

Mary appears in 22 solutions.
Bruce appears in 31 solutions.
Ciara appears in 48 solutions.
Anita appears in 56 solutions.
Jim appears in 56 solutions.
Peter appears in 61 solutions.
Juan appears in 74 solutions.
Jane appears in ALL solutions

The number of solutions that Ciara is not in is: 29
The number of solutions that Juan is not in is: 3
The number of solutions that neither is in is: 1
The number of solutions with BOTH in it is: 46


## Solution where neither Ciara nor Juan are needed

In [8]:
for i, solution in enumerate(unique_solutions2):
    if "Ciara" not in solution.values() and "Juan" not in solution.values():
        print(f"Solution {i+1}:")
        for position, hire in solution.items():
            print(f"Position {position}: {hire}")
        print()     

Solution 24:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Peter
Position Python_1: Jane
Position Python_2: Peter
Position Database: Jane
Position Systems: Jim
Position Web: Anita



## Solution without CSP

# DV - Visualizations

## Scenario 1

In [5]:
import pandas as pd

# Convert the list of solutions into a DataFrame
df_scenario1 = pd.DataFrame(unique_solutions1)

df_scenario1.index.name = "Hiring option"
df_scenario1.reset_index(inplace=True)
# Remove the index name
df_scenario1.set_index("Hiring option", inplace=True)

df_scenario1.columns.name = df_scenario1.index.name
df_scenario1.index.name = None
df_scenario1.index = range(1, len(df_scenario1) + 1)


df_scenario1

Hiring option,AI_1,AI_2,Python_1,Database,Python_2,Systems,Web
1,Anita,Jim,Ciara,Jane,Jane,Jim,Anita
2,Jim,Juan,Ciara,Jane,Jane,Jim,Juan


## Scenario 2

In [6]:
df_scenario2 = pd.DataFrame(unique_solutions2)

df_scenario2.index.name = "Hiring option"
df_scenario2.reset_index(inplace=True)

df_scenario2.set_index("Hiring option", inplace=True)

df_scenario2.columns.name = df_scenario2.index.name
df_scenario2.index.name = None
df_scenario2.index = range(1, len(df_scenario2) + 1)


df_scenario2

Hiring option,AI_1,AI_2,AI_3,Python_1,Python_2,Database,Systems,Web
1,Anita,Jim,Juan,Bruce,Ciara,Jane,Bruce,Juan
2,Anita,Jim,Juan,Bruce,Ciara,Jane,Bruce,Anita
3,Anita,Jim,Juan,Bruce,Ciara,Jane,Jim,Juan
4,Anita,Jim,Juan,Bruce,Ciara,Jane,Jim,Anita
5,Anita,Jim,Juan,Bruce,Jane,Jane,Bruce,Juan
...,...,...,...,...,...,...,...,...
73,Jim,Juan,Peter,Ciara,Peter,Jane,Mary,Mary
74,Jim,Juan,Peter,Ciara,Peter,Jane,Mary,Juan
75,Jim,Juan,Peter,Ciara,Peter,Jane,Jim,Anita
76,Jim,Juan,Peter,Ciara,Peter,Jane,Jim,Mary


## Frequency of each person as a hiring option

In [7]:
df_scenario2_frequency = pd.DataFrame(person_counts.items(), columns=["Hire Prospect", "Appears in solution (Times)"])
df_scenario2_frequency.sort_values(by="Appears in solution (Times)", ascending=False)

df_scenario2_frequency = df_scenario2_frequency.set_index("Hire Prospect")
df_scenario2_frequency.columns.name = df_scenario2_frequency.index.name
df_scenario2_frequency.index.name = None

df_scenario2_frequency

Hire Prospect,Appears in solution (Times)
Mary,22
Bruce,31
Ciara,48
Anita,56
Jim,56
Peter,61
Juan,74
Jane,77
